## Ezyme-Subtrate Kinetics Rule Creator


This notebook uses the Boolean_rules_creator tool available to generate
rules for a boolean network describing Michaelis-Menten enzyme-substrate kinetics. For more information on the tool and installation instructions, view the GitHub page at https://github.com/LoLab-VU/Boolean_rules_creator .

>More details on the algorithm and a description of the model are available in the preprint at https://www.biorxiv.org/content/10.1101/2020.12.15.422874v1.full (Prugger et. al. 2020).

### Importing libraries


In [1]:
from rule_creator import creating_rules
import json
import sympy
from sympy.logic.boolalg import Xor
from sympy import symbols

### Loading the input file

For the MM model, there is only one possibly steady state outcome, so the Boolean ruleset can be generation without optimization. The list  In this example, only the rule_creator function is used to recreate the forward, backwards, and expert-guided rulesets.

The input file for this model is 'MM_steady_states.json', which contains the possible initial boolean states for the four species: Enzyme (EN), Substrate (S), Enzyme-Substrate complex (ES), and Product (P). For this script, these states are always referenced in this order.

In [2]:
fn = 'ES_steady_states.json'
En, Su, ES, Pr = symbols('En Su ES Pr')
symbols_list = ['En', 'Su', 'ES', 'Pr']

The input file contains pairs of intial and steady states followed by the frequency
with which the intial state leads to that final state. 

In [3]:
with open(fn) as fs:
    data = json.load(fs)
    fs.close()
print(data.items())

dict_items([('[0, 0, 0, 0]', [[[0, 0, 0, 0], 100]]), ('[0, 0, 0, 1]', [[[0, 0, 0, 1], 100]]), ('[0, 0, 1, 0]', [[[1, 0, 0, 1], 100]]), ('[0, 0, 1, 1]', [[[1, 0, 0, 1], 100]]), ('[0, 1, 0, 0]', [[[0, 1, 0, 0], 100]]), ('[0, 1, 0, 1]', [[[0, 1, 0, 1], 100]]), ('[0, 1, 1, 0]', [[[1, 0, 0, 1], 100]]), ('[0, 1, 1, 1]', [[[1, 0, 0, 1], 100]]), ('[1, 0, 0, 0]', [[[1, 0, 0, 0], 100]]), ('[1, 0, 0, 1]', [[[1, 0, 0, 1], 100]]), ('[1, 0, 1, 0]', [[[1, 0, 0, 1], 100]]), ('[1, 0, 1, 1]', [[[1, 0, 0, 1], 100]]), ('[1, 1, 0, 0]', [[[1, 0, 0, 1], 100]]), ('[1, 1, 0, 1]', [[[1, 0, 0, 1], 100]]), ('[1, 1, 1, 0]', [[[1, 0, 0, 1], 100]]), ('[1, 1, 1, 1]', [[[1, 0, 0, 1], 100]])])


### Generating network rules

The function creating_rules will take these input states and generate a list of rules which contain
the possible transitions to describe the boolean network. For this model, the first three states, as seen below, can only progress to their respective steady states due to the lack of reactive species. For all other intial states the only possible steady state is [1,0,0,1].

In [4]:
str_rules, simple_rulelist, fs_cpp_name = creating_rules(fn,symbols_list,backwardpaths=0)

	
data:  {'[0, 0, 0, 0]': [[[0, 0, 0, 0], 100]], '[0, 0, 0, 1]': [[[0, 0, 0, 1], 100]], '[0, 0, 1, 0]': [[[1, 0, 0, 1], 100]], '[0, 0, 1, 1]': [[[1, 0, 0, 1], 100]], '[0, 1, 0, 0]': [[[0, 1, 0, 0], 100]], '[0, 1, 0, 1]': [[[0, 1, 0, 1], 100]], '[0, 1, 1, 0]': [[[1, 0, 0, 1], 100]], '[0, 1, 1, 1]': [[[1, 0, 0, 1], 100]], '[1, 0, 0, 0]': [[[1, 0, 0, 0], 100]], '[1, 0, 0, 1]': [[[1, 0, 0, 1], 100]], '[1, 0, 1, 0]': [[[1, 0, 0, 1], 100]], '[1, 0, 1, 1]': [[[1, 0, 0, 1], 100]], '[1, 1, 0, 0]': [[[1, 0, 0, 1], 100]], '[1, 1, 0, 1]': [[[1, 0, 0, 1], 100]], '[1, 1, 1, 0]': [[[1, 0, 0, 1], 100]], '[1, 1, 1, 1]': [[[1, 0, 0, 1], 100]]}
	
steady state: [0, 0, 0, 0]
	(0, 0, 0, 0) <- 100
	
	
steady state: [0, 0, 0, 1]
	(0, 0, 0, 1) <- 100
	
	
steady state: [1, 0, 0, 1]
	(0, 0, 1, 0) <- 100
	(0, 0, 1, 1) <- 100
	(0, 1, 1, 0) <- 100
	(0, 1, 1, 1) <- 100
	(1, 0, 0, 1) <- 100
	(1, 0, 1, 0) <- 100
	(1, 0, 1, 1) <- 100
	(1, 1, 0, 0) <- 100
	(1, 1, 0, 1) <- 100
	(1, 1, 1, 0) <- 100
	(1, 1, 1, 1) <- 100
	


Looking at the resulting rule list:

In [5]:
print('The string rules describe the full ruleset in formal logical terms, showing what states can activate that species.')
print('\nString rules:')
print(str_rules)

print('The simple rulelist show the string rules in their boolean form; this conveys the same information as above.')
print('\nSimple rulelist')
print(simple_rulelist)

The string rules describe the full ruleset in formal logical terms, showing what states can activate that species.

String rules:
1: En* = Xor((( not En and Su and ES and Pr) or ( not En and Su and ES and  not Pr) or ( not En and  not Su and ES and  not Pr) or ( not En and  not Su and ES and Pr)), En)
1: Su* = Xor(((En and Su and  not ES and Pr) or (En and Su and ES and  not Pr) or ( not En and Su and ES and Pr) or (En and Su and ES and Pr) or ( not En and Su and ES and  not Pr)), Su)
1: ES* = Xor(((En and  not Su and ES and Pr) or (En and Su and ES and  not Pr) or (En and Su and ES and Pr)), ES)
1: Pr* = Xor(((En and Su and ES and  not Pr) or (En and  not Su and ES and  not Pr) or (En and Su and  not ES and  not Pr) or ( not En and  not Su and ES and  not Pr) or ( not En and Su and ES and  not Pr)), Pr)

The simple rulelist show the string rules in their boolean form; this conveys the same information as above.

Simple rulelist
[[(0, 1, 1, 1), (0, 1, 1, 0), (0, 0, 1, 0), (0, 0, 1, 1)]


With the input variable backwardspath set as 1, we generate the network with all possible backwards paths available
for transitions. The simple rulelist output describes the resulting network. The example below (from Figure 3) shows the network formed with the initial state [1,1,0,0].


![alt text](bw_1100_4.png "ES-B Network")

### Generating simplified rules from manuscript

In order to simplify the rules further using Sympy, the string rules must first be converted to formal logic notation

In [6]:
str_rules_list = str_rules.split('\n')
str_rules_list = str_rules_list[0:4]
count = 0

for n in str_rules_list: 
    str_rules_list[count] = n.strip('1: '+symbols_list[count]+'* = ') 
    count +=1

sympy_input = []
for k in range(len(str_rules_list)):
    sympy_input.append(str_rules_list[k].replace('and', '&').replace(' or',' |').replace('not ', '~'))

print(sympy_input)

['Xor((( ~En & Su & ES & Pr) | ( ~En & Su & ES &  ~Pr) | ( ~En &  ~Su & ES &  ~Pr) | ( ~En &  ~Su & ES & Pr)), En)', 'Xor(((En & Su &  ~ES & Pr) | (En & Su & ES &  ~Pr) | ( ~En & Su & ES & Pr) | (En & Su & ES & Pr) | ( ~En & Su & ES &  ~Pr)), Su)', 'Xor(((En &  ~Su & ES & Pr) | (En & Su & ES &  ~Pr) | (En & Su & ES & Pr)), ES)', 'Xor(((En & Su & ES &  ~Pr) | (En &  ~Su & ES &  ~Pr) | (En & Su &  ~ES &  ~Pr) | ( ~En &  ~Su & ES &  ~Pr) | ( ~En & Su & ES &  ~Pr)), Pr)']


Using Sympy to simplify the Xor statement for each species yields the rules for the ES-B network as seen in the paper

In [7]:
print('Rules after simplifcation with Sympy')
#exp_EN = sympy.simplify(sympy.sympify(sympy_input[0][1]))

print('EN* = ' + str(sympy.to_dnf(sympy_input[0], simplify=True)))
print('S* = ' + str(sympy.to_dnf(sympy_input[1], simplify=True)))
print('ES* = ' + str(sympy.to_dnf(sympy_input[2], simplify=True)))
print('P* = ' + str(sympy.to_dnf(sympy_input[3], simplify=True)))


Rules after simplifcation with Sympy
EN* = ES | En
S* = (Su & ~ES & ~En) | (Su & ~ES & ~Pr)
ES* = (ES & ~En) | (ES & ~Pr & ~Su)
P* = ES | Pr | (En & Su)


In [4]:
addlist = [((1, 1, 1, 0), 0), ((1, 1, 0, 0), 2)]

str_rules_expert_fw, simple_rulelist_expert_fw, fs_cpp_name_expert_fw = creating_rules(fn,symbols_list,backwardpaths=0,addlist=addlist)

	
data:  {'[0, 0, 0, 0]': [[[0, 0, 0, 0], 100]], '[0, 0, 0, 1]': [[[0, 0, 0, 1], 100]], '[0, 0, 1, 0]': [[[1, 0, 0, 1], 100]], '[0, 0, 1, 1]': [[[1, 0, 0, 1], 100]], '[0, 1, 0, 0]': [[[0, 1, 0, 0], 100]], '[0, 1, 0, 1]': [[[0, 1, 0, 1], 100]], '[0, 1, 1, 0]': [[[1, 0, 0, 1], 100]], '[0, 1, 1, 1]': [[[1, 0, 0, 1], 100]], '[1, 0, 0, 0]': [[[1, 0, 0, 0], 100]], '[1, 0, 0, 1]': [[[1, 0, 0, 1], 100]], '[1, 0, 1, 0]': [[[1, 0, 0, 1], 100]], '[1, 0, 1, 1]': [[[1, 0, 0, 1], 100]], '[1, 1, 0, 0]': [[[1, 0, 0, 1], 100]], '[1, 1, 0, 1]': [[[1, 0, 0, 1], 100]], '[1, 1, 1, 0]': [[[1, 0, 0, 1], 100]], '[1, 1, 1, 1]': [[[1, 0, 0, 1], 100]]}
	
steady state: [0, 0, 0, 0]
	(0, 0, 0, 0) <- 100
	
	
steady state: [0, 0, 0, 1]
	(0, 0, 0, 1) <- 100
	
	
steady state: [1, 0, 0, 1]
	(0, 0, 1, 0) <- 100
	(0, 0, 1, 1) <- 100
	(0, 1, 1, 0) <- 100
	(0, 1, 1, 1) <- 100
	(1, 0, 0, 1) <- 100
	(1, 0, 1, 0) <- 100
	(1, 0, 1, 1) <- 100
	(1, 1, 0, 0) <- 100
	(1, 1, 0, 1) <- 100
	(1, 1, 1, 0) <- 100
	(1, 1, 1, 1) <- 100
	


In [7]:
str_rules_list = str_rules_expert_fw.split('\n')
str_rules_list = str_rules_list[0:4]
count = 0

for n in str_rules_list: 
    str_rules_list[count] = n.strip('1: '+symbols_list[count]+'* = ') 
    count +=1

sympy_input = []
for k in range(len(str_rules_list)):
    sympy_input.append(str_rules_list[k].replace('and', '&').replace(' or',' |').replace('not ', '~'))

print(sympy_input)

['Xor(((En & Su & ES &  ~Pr) | ( ~En & Su & ES & Pr) | ( ~En &  ~Su & ES &  ~Pr) | ( ~En & Su & ES &  ~Pr) | ( ~En &  ~Su & ES & Pr)), En)', 'Xor(((En & Su &  ~ES & Pr) | (En & Su & ES &  ~Pr) | ( ~En & Su & ES & Pr) | (En & Su & ES & Pr) | ( ~En & Su & ES &  ~Pr)), Su)', 'Xor(((En &  ~Su & ES & Pr) | (En & Su &  ~ES &  ~Pr) | (En & Su & ES &  ~Pr) | (En & Su & ES & Pr)), ES)', 'Xor(((En & Su & ES &  ~Pr) | (En &  ~Su & ES &  ~Pr) | (En & Su &  ~ES &  ~Pr) | ( ~En &  ~Su & ES &  ~Pr) | ( ~En & Su & ES &  ~Pr)), Pr)']


In [8]:
print('Rules after simplifcation with Sympy')
#exp_EN = sympy.simplify(sympy.sympify(sympy_input[0][1]))

print('EN* = ' + str(sympy.to_dnf(sympy_input[0], simplify=True)))
print('S* = ' + str(sympy.to_dnf(sympy_input[1], simplify=True)))
print('ES* = ' + str(sympy.to_dnf(sympy_input[2], simplify=True)))
print('P* = ' + str(sympy.to_dnf(sympy_input[3], simplify=True)))


Rules after simplifcation with Sympy
EN* = (En & Pr) | (ES & ~En) | (En & ~ES) | (En & ~Su)
S* = (Su & ~ES & ~En) | (Su & ~ES & ~Pr)
ES* = (ES & ~En) | (ES & ~Pr & ~Su) | (En & Su & ~ES & ~Pr)
P* = ES | Pr | (En & Su)


In [4]:
blacklist = [((1, 0, 1, 1), 0), ((1, 0, 1, 0), 0), ((1, 1, 1, 1), 0),
             ((1, 0, 1, 1), 1), ((1, 0, 1, 0), 1), ((0, 0, 1, 0), 1), ((0, 0, 1, 1), 1),
             ((1, 1, 0, 1), 2),
             ((1, 0, 1, 1), 3), ((1, 1, 0, 1), 3), ((0, 1, 1, 1), 3), ((1, 1, 1, 1), 3), ((0, 0, 1, 1), 3)]

str_rules_expert_bw, simple_rulelist_expert_bw, fs_cpp_name_expert_bw = creating_rules(fn,symbols_list,backwardpaths=1,blacklist=blacklist)

	
data:  {'[0, 0, 0, 0]': [[[0, 0, 0, 0], 100]], '[0, 0, 0, 1]': [[[0, 0, 0, 1], 100]], '[0, 0, 1, 0]': [[[1, 0, 0, 1], 100]], '[0, 0, 1, 1]': [[[1, 0, 0, 1], 100]], '[0, 1, 0, 0]': [[[0, 1, 0, 0], 100]], '[0, 1, 0, 1]': [[[0, 1, 0, 1], 100]], '[0, 1, 1, 0]': [[[1, 0, 0, 1], 100]], '[0, 1, 1, 1]': [[[1, 0, 0, 1], 100]], '[1, 0, 0, 0]': [[[1, 0, 0, 0], 100]], '[1, 0, 0, 1]': [[[1, 0, 0, 1], 100]], '[1, 0, 1, 0]': [[[1, 0, 0, 1], 100]], '[1, 0, 1, 1]': [[[1, 0, 0, 1], 100]], '[1, 1, 0, 0]': [[[1, 0, 0, 1], 100]], '[1, 1, 0, 1]': [[[1, 0, 0, 1], 100]], '[1, 1, 1, 0]': [[[1, 0, 0, 1], 100]], '[1, 1, 1, 1]': [[[1, 0, 0, 1], 100]]}
	
steady state: [0, 0, 0, 0]
	(0, 0, 0, 0) <- 100
	
	
steady state: [0, 0, 0, 1]
	(0, 0, 0, 1) <- 100
	
	
steady state: [1, 0, 0, 1]
	(0, 0, 1, 0) <- 100
	(0, 0, 1, 1) <- 100
	(0, 1, 1, 0) <- 100
	(0, 1, 1, 1) <- 100
	(1, 0, 0, 1) <- 100
	(1, 0, 1, 0) <- 100
	(1, 0, 1, 1) <- 100
	(1, 1, 0, 0) <- 100
	(1, 1, 0, 1) <- 100
	(1, 1, 1, 0) <- 100
	(1, 1, 1, 1) <- 100
	


In [5]:
str_rules_list = str_rules_expert_bw.split('\n')
str_rules_list = str_rules_list[0:4]
count = 0

for n in str_rules_list: 
    str_rules_list[count] = n.strip('1: '+symbols_list[count]+'* = ') 
    count +=1

sympy_input = []
for k in range(len(str_rules_list)):
    sympy_input.append(str_rules_list[k].replace('and', '&').replace(' or',' |').replace('not ', '~'))

print(sympy_input)

['Xor(((En & Su & ES &  ~Pr) | ( ~En & Su & ES & Pr) | ( ~En &  ~Su & ES &  ~Pr) | ( ~En & Su & ES &  ~Pr) | ( ~En &  ~Su & ES & Pr)), En)', 'Xor(((En & Su &  ~ES & Pr) | (En & Su & ES &  ~Pr) | ( ~En & Su & ES & Pr) | (En & Su & ES & Pr) | ( ~En & Su & ES &  ~Pr)), Su)', 'Xor(((En &  ~Su & ES & Pr) | (En & Su &  ~ES &  ~Pr) | (En & Su & ES &  ~Pr) | (En & Su & ES & Pr)), ES)', 'Xor(((En & Su & ES &  ~Pr) | (En &  ~Su & ES &  ~Pr) | (En & Su &  ~ES &  ~Pr) | ( ~En &  ~Su & ES &  ~Pr) | ( ~En & Su & ES &  ~Pr)), Pr)']


In [6]:
print('Rules after simplifcation with Sympy')
#exp_EN = sympy.simplify(sympy.sympify(sympy_input[0][1]))

print('EN* = ' + str(sympy.to_dnf(sympy_input[0], simplify=True)))
print('S* = ' + str(sympy.to_dnf(sympy_input[1], simplify=True)))
print('ES* = ' + str(sympy.to_dnf(sympy_input[2], simplify=True)))
print('P* = ' + str(sympy.to_dnf(sympy_input[3], simplify=True)))


Rules after simplifcation with Sympy
EN* = (En & Pr) | (ES & ~En) | (En & ~ES) | (En & ~Su)
S* = (Su & ~ES & ~En) | (Su & ~ES & ~Pr)
ES* = (ES & ~En) | (ES & ~Pr & ~Su) | (En & Su & ~ES & ~Pr)
P* = ES | Pr | (En & Su)
